In [1]:
import pandas as pd
import os
os.chdir("/home/yangkai/codegen-detection")

## APPS

In [2]:
hum_data = pd.read_pickle("dataset/mbpp_hum_test100.pkl")
gen_data = pd.read_pickle("dataset/mbpp_gen_test100_gpt4.pkl")

# hum_data = pd.read_pickle("dataset/csn_1k_hum_test.pkl")
# gen_data = pd.read_pickle("dataset/csn_gpt_gen_test.pkl")
hum_data['label'] = 0
gen_data['label'] = 1

In [3]:
from simcse.simcse import RobertaForCL
from transformers import AutoTokenizer
device = 1
model_args = {'pooler_type':'cls','temp':0.1,'mlp_only_train':True}
# model = RobertaForCL.from_pretrained("simcse/ckpt_gcb4_csn/best_model",**model_args)
model = RobertaForCL.from_pretrained("simcse/ckpt_gcb4_codenet_py_no_overlap/best_model",**model_args)

# model_args = {'pooler_type':'avg','temp':0.1,'mlp_only_train':True}
# model = RobertaForCL.from_pretrained("microsoft/graphcodebert-base",**model_args)

tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")

model = model.to(device)

/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/yangkai/anaconda3/envs/torch20-cu117/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so.11.0'), PosixPath('/home/yangkai/anaconda3/envs/torch20-cu117/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


[2024-01-26 17:20:35,549] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
{'pooler_type': 'cls', 'temp': 0.1, 'mlp_only_train': True}


In [4]:
import torch
from tqdm import tqdm
bsz = 32

@torch.inference_mode()
def pipeline(texts):
    inputs = tokenizer(texts,return_tensors='pt',padding=True,add_special_tokens=True,truncation=True)
    inputs = {k:v.to(device) for k,v in inputs.items()}
    sent_embed = model(**inputs,sent_emb=True).pooler_output

    return sent_embed

@torch.inference_mode()
def batch_encode(texts):
    embeds = []
    for i in tqdm(range(0,len(texts),bsz)):
        chunk = texts[i:i+bsz]
        embeds.append(pipeline(chunk))
    
    return torch.cat(embeds,dim=0)

In [31]:
# gen_resample_keys = ['gpt_resample-nocot_code_cleaned1','gpt_resample-nocot_code_cleaned2']
# gen_resample_keys = ['vicuna_resampled_code_cleaned','vicuna_resampled_code_cleaned2'] + [f'gpt_resample_code_cleaned{i}' for i in range(3,9)]
gen_resample_keys = [f'gpt_resample_code_cleaned{i}' for i in range(1,9)]
# gen_resample_keys = [f'vicuna7b_resample_code_cleaned{i}' for i in range(1,9)]
gen_orig_key = 'extracted_full_func'

hum_resample_keys = ['vicuna_resampled_code_cleaned','vicuna_resampled_code_cleaned2'] + [f'gpt_resample_code_cleand{i}' for i in range(3,9)]
# hum_resample_keys = [f'gpt_resample_code_cleaned{i}' for i in range(1,9)]
# hum_resample_keys = [f'vicuna7b_resample_code_cleaned{i}' for i in range(1,9)]
hum_orig_key = 'extracted_full_func'

In [37]:
for name in gen_data.columns:
    print(name)

problem_id
question
solutions
difficulty
url
starter_code
apps_split
code_source
solution
extracted_full_func
extracted_full_func_word_prob_CodeLlama-13b-Instruct-hf
extracted_full_func_word_rank_CodeLlama-13b-Instruct-hf
extracted_full_func_entropy_CodeLlama-13b-Instruct-hf
extracted_full_func_word_prob_starchat-alpha
extracted_full_func_word_rank_starchat-alpha
extracted_full_func_entropy_starchat-alpha
extracted_full_func_word_prob_incoder-6B
extracted_full_func_word_rank_incoder-6B
extracted_full_func_entropy_incoder-6B
extracted_full_func_word_prob_codegen-2B-mono
extracted_full_func_word_rank_codegen-2B-mono
extracted_full_func_entropy_codegen-2B-mono
extracted_full_func_word_prob_PolyCoder-160M
extracted_full_func_word_rank_PolyCoder-160M
extracted_full_func_entropy_PolyCoder-160M
label


In [87]:
import json
import torch
def load_openai_embed(path):
    response = [json.loads(line.strip()) for line in open(path,'r').readlines()]
    all_embeds = [item['data'][0]['embedding'] for item in response]
    return torch.FloatTensor(all_embeds)

hum_orig = load_openai_embed("prompt_gpt/apps_hum_test100_orig_openaiemb.jsonl")
gen_orig = load_openai_embed("prompt_gpt/apps_gen_test100_orig_openaiemb.jsonl")
orig = torch.cat([gen_orig,hum_orig])

hum_regpt1 = load_openai_embed("prompt_gpt/apps_hum_test100_re3_openaiemb.jsonl")
gen_regpt1 = load_openai_embed("prompt_gpt/apps_gen_test100_re3_openaiemb.jsonl")
regpt1 = torch.cat([gen_regpt1,hum_regpt1])

hum_regpt2 = load_openai_embed("prompt_gpt/apps_hum_test100_re4_openaiemb.jsonl")
gen_regpt2 = load_openai_embed("prompt_gpt/apps_gen_test100_re4_openaiemb.jsonl")
regpt2 = torch.cat([gen_regpt2,hum_regpt2])

hum_resc1 = load_openai_embed("prompt_gpt/apps_hum_test100_reStarcoder1_openaiemb.jsonl")
gen_resc1 = load_openai_embed("prompt_gpt/apps_gen_test100_reStarcoder1_openaiemb.jsonl")
resc1 = torch.cat([gen_resc1,hum_resc1])

hum_resc2 = load_openai_embed("prompt_gpt/apps_hum_test100_reStarcoder2_openaiemb.jsonl")
gen_resc2 = load_openai_embed("prompt_gpt/apps_gen_test100_reStarcoder2_openaiemb.jsonl")
resc2 = torch.cat([gen_resc2,hum_resc2])


gpt_sim1 = (orig * regpt1).sum(-1)
gpt_sim2 = (orig * regpt2).sum(-1)
gpt_sim = (gpt_sim1 + gpt_sim2) / 2

sc_sim1 = (orig * resc1).sum(-1)
sc_sim2 = (orig * resc2).sum(-1)
sc_sim = (sc_sim1 + sc_sim2) / 2

roc_auc_score([1] * 100 + [0] * 100, sc_sim)

0.6100000000000001

In [6]:
def get_sim(data,resample_keys,orig_key):
    orig_embed = batch_encode(data[orig_key].tolist())
    resample_embeds = []
    for key in resample_keys:
        resample_embeds.append(batch_encode(data[key].tolist()))
    
    all_sim = []
    normed_orig_embed = torch.nn.functional.normalize(orig_embed,p=2,dim=-1)
    for embed in resample_embeds:
        normed_embed = torch.nn.functional.normalize(embed,p=2,dim=-1)
        sim = (normed_orig_embed * normed_embed).sum(-1)
        all_sim.append(sim)
    
    return all_sim

In [32]:
hum_sim = get_sim(hum_data,hum_resample_keys,hum_orig_key)
gen_sim = get_sim(gen_data,gen_resample_keys,gen_orig_key)

100%|██████████| 4/4 [00:00<00:00,  5.79it/s]


In [35]:
from sklearn.metrics import roc_auc_score
all_data = pd.concat([gen_data,hum_data])
all_sim = [torch.concat([sim1,sim2]) for sim1,sim2 in zip(gen_sim,hum_sim)]
m = 2
score = sum([all_sim[i] for i in range(m)]) / m
# score = (all_sim[0] + all_sim[1]) / 2

label = all_data['label'].tolist()
score = score.tolist()

roc_auc = roc_auc_score(label,score)
roc_auc

0.8202

In [58]:
# no cot resample apps 81.21
# cot resample apps 78.49

# direct gen code 80.38

0.5259921259842519

In [ ]:
# ident replaced 0.2 64.57
# ident replaced 0.5 52.00
# ident replaced 0.1 74.08

In [ ]:
# APPS GCB avg_last GPT-3.5-turbo
# m = 2 70.23 67.91(100)
# m = 4 71.53
# m = 8 72.15

# APPS GCB avg_last StarCoder
# m = 2 66.98 65.22(100)
# m = 4 66.19
# m = 8 64.85

In [ ]:
# MBPP GCB avg_last GPT-3.5-turbo
# m = 2 75.89
# m = 4 77.09
# m = 8 76.94

# MBPP GCB avg_last StarCoder
# m = 2 80.02
# m = 4 82.60
# m = 8 81.04